<a href="https://colab.research.google.com/github/ares97/BlockRush/blob/master/102flower_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
# download PyTorch and other used dependencies
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

!pip install --no-cache-dir -I pillow
!pip install Pillow==4.1.1
!pip install PIL
!pip install image

!conda install -c menpo wget --yes
!sudo apt install wget


    100% |████████████████████████████████| 2.0MB 2.0MB/s 
    100% |████████████████████████████████| 5.6MB 7.3MB/s 
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
  Found existing installation: Pillow 5.4.1
    Uninstalling Pillow-5.4.1:
      Successfully uninstalled Pillow-5.4.1
  Could not find a version that satisfies the requirement PIL (from versions: )
No matching distribution found for PIL
/bin/bash: conda: command not found
Reading package lists... Done
Building dependency tree       
Reading state information... Done
wget is already the newest version (1.19.4-1ubuntu2.1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.


In [0]:
# download helper for showing tensors as img etc.
!wget -cq https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/intro-to-pytorch/helper.py
# Download and unzip data
!rm -rf flower_data
!wget -cq https://github.com/udacity/pytorch_challenge/raw/master/cat_to_name.json
!wget -cq https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip
!unzip -qq flower_data.zip



In [12]:
import torch

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    device = 'cpu'
else:
    device = 'cuda'

print("Training on", device)

Training on cuda


In [0]:
# Prepare and augument the data

from torchvision import transforms
from torchvision import datasets
import json


data_dir = "flower_data"

training_transforms = transforms.Compose([transforms.RandomRotation(45),                                                                                                             
                                          transforms.RandomHorizontalFlip(),                                               
                                          transforms.RandomCrop(224),
                                          transforms.RandomResizedCrop(224), 
                                          transforms.ColorJitter(hue=.05, saturation=.05),
                                          transforms.ToTensor(),
                                          transforms.Normalize((0.485,0.456,0.406),
                                                               (0.229,0.224,0.225))])    
                                                                                                                                         
                                                                                                                         
validation_transforms = transforms.Compose([transforms.Resize(256), 
                                            transforms.CenterCrop(224), 
                                            transforms.ToTensor(),
                                            transforms.Normalize((0.485,0.456,0.406),
                                                                 (0.229,0.224,0.225))])


training_dataset = datasets.ImageFolder(data_dir + '/train', transform=training_transforms)
validation_dataset = datasets.ImageFolder(data_dir + '/valid', transform=validation_transforms)


training_dataloader = torch.utils.data.DataLoader(training_dataset, batch_size=64, shuffle=True)
validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=64)

### Load label mapping file
cat_to_name = json.load(open('cat_to_name.json'))

In [19]:
"""
Utility function for computing output of convolutions
takes a tuple of (h,w) and returns a tuple of (h,w)
"""
def conv_output_shape(h_w, kernel_size=1, stride=1, pad=0, dilation=1):
    from math import floor
    if type(kernel_size) is not tuple:
        kernel_size = (kernel_size, kernel_size)
    h = floor( ((h_w[0] + (2 * pad) - ( dilation * (kernel_size[0] - 1) ) - 1 )/ stride) + 1)
    w = floor( ((h_w[1] + (2 * pad) - ( dilation * (kernel_size[1] - 1) ) - 1 )/ stride) + 1)
    return h, w
  
conv_output_shape([224,224], 8, 2, 3)

(112, 112)

In [0]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
  def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 64, kernel_size=8, stride=2, padding=3, bias=False)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=4, stride=2, padding=3, bias=False) 
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=1, bias=False)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=2, stride=2, padding=1, bias=False)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        self.fc1 = nn.Linear(5*5*64, 1024)
        self.fc4 = nn.Linear(1024, 512)
        self.fc5 = nn.Linear(512, 102)

        self.dropout = nn.Dropout(0.25)

  def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x))) # 56x56
        x = self.pool(F.relu(self.conv2(x))) # 15x15
        x = F.relu(self.conv3(x))            # 8x8
        x = F.relu(self.conv4(x))            # 5x5 
   #     x = self.pool(F.relu(self.conv5(x)))

        # flatten image input
        x = x.view(-1, 5*5*64)

        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc4(x))
        x = self.dropout(x)
        x = F.log_softmax(self.fc5(x), dim=1)
  
        return x
      


In [25]:
from torchvision import models
import torch.optim as optim

model = Net()

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)

model.to(device)

Net(
  (conv1): Conv2d(3, 64, kernel_size=(8, 8), stride=(2, 2), padding=(3, 3), bias=False)
  (conv2): Conv2d(64, 64, kernel_size=(4, 4), stride=(2, 2), padding=(3, 3), bias=False)
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (conv4): Conv2d(64, 64, kernel_size=(2, 2), stride=(2, 2), padding=(1, 1), bias=False)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1600, out_features=1024, bias=True)
  (fc4): Linear(in_features=1024, out_features=512, bias=True)
  (fc5): Linear(in_features=512, out_features=102, bias=True)
  (dropout): Dropout(p=0.25)
)

In [33]:
import numpy as np

epochs = 20
best_acc = np.NINF

train_losses = []
train_accs = []
valid_losses = []
valid_accs = []

for epoch in range(epochs):

  #set model for training
  model.train()
  train_loss = 0
  train_acc = 0
  
  # training loop
  for inputs, labels in training_dataloader:
    inputs, labels = inputs.to(device), labels.to(device)
    
    # train flow
    optimizer.zero_grad()
    logps = model.forward(inputs)
    loss = criterion(logps, labels)
    loss.backward()
    optimizer.step()
    
    # calculate train loss and acc
    ps = torch.exp(logps)
    top_ps, top_class = ps.topk(1, dim=1)
    equals = top_class == labels.view(*top_class.shape)
    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
    train_loss += loss.item()
    
    # calculate avg loss and acc and append them to lists
    train_losses.append(train_loss / len(training_dataloader))
    train_accs.append(train_acc / len(training_dataloader))
    

    
  # set model for validation
  model.eval()
  valid_acc = 0
  valid_loss = 0
 
  # validation loop
  for inputs, labels in validation_dataloader:
    inputs, labels = inputs.to(device), labels.to(device)
    
    # calculate validation loss
    logps = model.forward(inputs)
    loss = criterion(logps, labels)
    valid_loss += loss.item()
    
    # calculate validation acc
    ps = torch.exp(logps)
    top_ps, top_class = ps.topk(1, dim=1)
    equals = top_class == labels.view(*top_class.shape)
    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
  
  
  
  # calculate avg loss and acc and append them to lists
  valid_losses.append(valid_loss / len(validation_dataloader))
  valid_accs.append(accuracy / len(validation_dataloader))
  
  print("Epoch {}/{}.. ".format(epoch+1, epochs),
          "Training loss: {:.3f}.. ".format(train_losses[epoch]),  
          "Training acc: {:.3f}.. ".format(train_acc[epoch]),  
          "Validation loss: {:.3f}..".format(valid_loss[epoch]),
          "Validation accuracy: {:.3f}..".format(valid_acc[epoch]))
  
  
  if accuracy > best_acc:
    best_acc = accuracy / len(validation_dataloader)
    torch.save(model.state_dict(), 'best_acc_model.pth')
    print("better acc: {:.3f} -> {:.3f}".format(best_acc, valid_accs[epoch]))
  
  

NameError: ignored